<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3ce6ed16a889288606577a71880d151183fda5084bc30281dafb07e54d1c31c3
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-08 09:23:15
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 8.89 K (0.06%)
Current PnL: -20.45 L (-13.57%)
CY Booked + Current PnL: -9.01 L (-5.98%)
-------------------
Total profit:  1.37 L
Total loss:  -21.82 L
-------------------
Total Booked + Current PnL: 17.97 L (14.53%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.11%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 87.95 L (62.32%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.99%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,67.0,H-MC,14.13,117602.0,17098.0,12007.0,2.22,17.01,10.21,28.96,79.0,1.42,0.84,56.76,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.60,40.0,M-SC,1.18,83408.0,-17369.0,17466.0,0.05,-17.24,20.94,0.09,245.0,-0.99,0.59,9.58,OX40N,NTT,DURABLES
19,CIPLA,1662.74,-22.18,44.0,H-LC,3.83,207486.0,2986.0,20313.0,0.08,1.46,9.79,11.39,10.0,0.15,1.48,10.54,X40N,ATH,PHARMA
14,BLUESTARCO,2080.00,8.56,45.0,H-MC,3.00,187410.0,22740.0,20596.0,-1.99,13.81,10.99,26.31,89.0,1.10,1.33,22.89,X40N,NTT,AC
57,RELIANCE,1533.00,-13.66,51.0,H-LC,3.26,215966.0,5600.0,23173.0,-0.03,2.66,10.73,13.68,37.0,0.24,1.54,19.63,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HONAUT,58357.33,-21.65,52.0,H-SC,2.12,110445.0,-17493.0,64632.0,3.08,-13.67,58.52,36.84,143.0,-0.27,0.79,13.07,X40N,ATH,ELECTRICAL
58,REPCOHOME,880.00,-57.53,66.0,H-SC,2.79,243646.0,-42859.0,300196.0,2.58,-14.96,123.21,89.82,134.0,-0.14,1.73,28.91,XY24,NTT,FINANCE
50,NATIONALUM,244.55,-41.18,67.0,H-MC,14.13,117602.0,17098.0,12007.0,2.22,17.01,10.21,28.96,79.0,1.42,0.84,56.76,MH,ATH,METALS
56,RELAXO,1176.00,-38.72,43.0,H-SC,1.76,83585.0,-61575.0,137506.0,2.01,-42.42,164.51,52.31,136.0,-0.45,0.59,11.88,X40N,NTT,FOOTWEAR
27,HAPPSTMNDS,1488.71,-24.78,25.0,H-SC,12.31,81144.0,-47021.0,157046.0,1.78,-36.69,193.54,85.85,132.0,-0.30,0.58,1.78,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.0,8.56,45.0,H-MC,3.00,187410.0,22740.0,20596.0,-1.99,13.81,10.99,26.31,89.0,1.10,1.33,22.89,X40N,NTT,AC
55,RAJOOENG,143.1,-41.82,40.0,H-SC,15.93,88280.0,-14220.0,54822.0,-1.37,-13.87,62.10,39.61,114.0,-0.26,0.63,7.82,AR,ATH,MISC
10,BANDHANBNK,400.0,-13.31,47.0,H-SC,2.96,217777.0,-60782.0,318216.0,-1.19,-21.82,146.12,92.42,151.0,-0.19,1.55,25.86,XY24,NTT,BANKS
84,VOLTAS,1530.0,-2.90,45.0,H-MC,1.27,203670.0,11928.0,25825.0,-1.05,6.22,12.68,19.69,99.0,0.46,1.45,13.44,XY25,NTT,AC
11,BANKINDIA,190.0,-25.47,65.0,H-MC,11.16,192142.0,12334.0,100087.0,-1.04,6.86,52.09,62.52,88.0,0.12,1.37,43.28,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1230.14,65.0,M-SC,21.52,180804.0,-339.0,45816.0,-0.04,-0.19,25.34,25.10,235.0,-0.01,1.29,44.90,XY24,NTT,HEALTHCARE
35,ICICIPRULI,790.0,-21.00,49.0,H-MC,1.86,135894.0,106.0,42644.0,-0.36,0.08,31.38,31.48,107.0,0.00,0.97,12.21,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.25,46.0,M-LC,5.70,150989.0,44.0,109407.0,0.58,0.03,72.46,72.51,53.0,0.00,1.07,6.05,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,62.0,H-SC,13.48,130194.0,-12285.0,31806.0,-0.41,-8.62,24.43,13.70,163.0,-0.39,0.93,53.66,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.60,40.0,M-SC,1.18,83408.0,-17369.0,17466.0,0.05,-17.24,20.94,0.09,245.0,-0.99,0.59,9.58,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.61,52.0,H-SC,2.03,223020.0,-46647.0,82986.0,0.54,-17.30,37.21,13.47,138.0,-0.56,1.59,13.40,XY24,NTT,PAINTS
18,CERA,9475.0,-23.15,36.0,H-SC,1.59,140231.0,-35672.0,77688.0,0.13,-20.28,55.40,23.89,149.0,-0.46,1.00,21.11,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.05,45.0,H-MC,6.48,102809.0,-29026.0,71247.0,0.40,-22.02,69.30,32.03,98.0,-0.41,0.73,15.15,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,53.0,M-MC,12.39,228819.0,3857.0,163789.0,0.20,1.71,71.58,74.53,192.0,0.02,1.63,34.52,XY24,BTT,STEEL
32,HINDZINC,730.22,31.71,68.0,M-LC,6.91,214390.0,9314.0,103250.0,0.60,4.54,48.16,54.89,52.0,0.09,1.53,29.73,X5K,ATH,METALS
14,BLUESTARCO,2080.00,8.56,45.0,H-MC,3.00,187410.0,22740.0,20596.0,-1.99,13.81,10.99,26.31,89.0,1.10,1.33,22.89,X40N,NTT,AC
84,VOLTAS,1530.00,-2.90,45.0,H-MC,1.27,203670.0,11928.0,25825.0,-1.05,6.22,12.68,19.69,99.0,0.46,1.45,13.44,XY25,NTT,AC
17,CAMS,5211.76,-6.27,44.0,H-SC,1.73,106436.0,4432.0,39488.0,0.02,4.34,37.10,43.06,122.0,0.11,0.76,22.21,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,67.0,H-MC,14.13,117602.0,17098.0,12007.0,2.22,17.01,10.21,28.96,79.0,1.42,0.84,56.76,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,65.0,H-MC,11.16,192142.0,12334.0,100087.0,-1.04,6.86,52.09,62.52,88.0,0.12,1.37,43.28,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,68.0,M-LC,6.91,214390.0,9314.0,103250.0,0.60,4.54,48.16,54.89,52.0,0.09,1.53,29.73,X5K,ATH,METALS
37,INDIAMART,4810.62,-55.20,34.0,H-SC,9.07,124879.0,1543.0,130086.0,0.41,1.25,104.17,106.72,119.0,0.01,0.89,24.92,AR,ATH,MISC
86,WIPRO,420.00,-14.25,46.0,M-LC,5.70,150989.0,44.0,109407.0,0.58,0.03,72.46,72.51,53.0,0.00,1.07,6.05,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.30,30.0,H-LC,10.14,279891.0,-35951.0,148622.0,-0.30,-11.38,53.10,35.67,5.0,-0.24,1.99,0.89,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-17.77,35.0,H-LC,5.73,207548.0,-44220.0,97423.0,0.24,-17.56,46.94,21.13,27.0,-0.45,1.48,11.64,X40,ATH,PAINTS
31,HINDUNILVR,2922.00,-10.67,40.0,H-LC,8.01,250810.0,-6253.0,41384.0,-0.36,-2.43,16.50,13.67,24.0,-0.15,1.78,17.48,XY25,NTT,FMCG
42,ITC,452.00,-39.61,41.0,H-LC,2.31,195673.0,-4465.0,25359.0,0.09,-2.23,12.96,10.44,4.0,-0.18,1.39,3.21,X40,NTT,FMCG
19,CIPLA,1662.74,-22.18,44.0,H-LC,3.83,207486.0,2986.0,20313.0,0.08,1.46,9.79,11.39,10.0,0.15,1.48,10.54,X40N,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.39,54.0,H-LC,1.96,161275.0,-24820.0,72300.0,-0.78,-13.34,44.83,25.51,15.0,-0.34,1.15,18.85,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,41.0,H-LC,2.31,195673.0,-4465.0,25359.0,0.09,-2.23,12.96,10.44,4.0,-0.18,1.39,3.21,X40,NTT,FMCG
48,LTIM,7230.2,-3.18,57.0,H-LC,2.42,239940.0,-6211.0,85419.0,1.20,-2.52,35.60,32.18,16.0,-0.07,1.71,32.61,X200,ATH,IT
51,NESTLEIND,1377.0,-10.90,48.0,H-LC,2.75,274201.0,8775.0,48013.0,-0.42,3.31,17.51,21.40,11.0,0.18,1.95,10.17,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.66,51.0,H-LC,3.26,215966.0,5600.0,23173.0,-0.03,2.66,10.73,13.68,37.0,0.24,1.54,19.63,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.30,30.0,H-LC,10.14,279891.0,-35951.0,148622.0,-0.30,-11.38,53.10,35.67,5.0,-0.24,1.99,0.89,X40N,ATH,FMCG
42,ITC,452.00,-39.61,41.0,H-LC,2.31,195673.0,-4465.0,25359.0,0.09,-2.23,12.96,10.44,4.0,-0.18,1.39,3.21,X40,NTT,FMCG
74,TCS,4406.34,-29.01,49.0,H-LC,12.20,282846.0,-63110.0,131354.0,1.19,-18.24,46.44,19.73,1.0,-0.48,2.01,4.18,X40,ATH,IT
80,UNITDSPR,1644.00,-11.66,54.0,H-LC,6.75,231426.0,-2718.0,51353.0,-0.47,-1.16,22.19,20.77,86.0,-0.05,1.65,5.68,X40N,NTT,BREWERIES
1,ABB,7934.00,-40.35,46.0,H-LC,7.50,249168.0,-12451.0,131660.0,-0.55,-4.76,52.84,45.57,7.0,-0.09,1.77,6.02,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.39,54.0,H-LC,1.96,161275.0,-24820.0,72300.0,-0.78,-13.34,44.83,25.51,15.0,-0.34,1.15,18.85,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,41.0,H-LC,2.31,195673.0,-4465.0,25359.0,0.09,-2.23,12.96,10.44,4.0,-0.18,1.39,3.21,X40,NTT,FMCG
19,CIPLA,1662.74,-22.18,44.0,H-LC,3.83,207486.0,2986.0,20313.0,0.08,1.46,9.79,11.39,10.0,0.15,1.48,10.54,X40N,ATH,PHARMA
5,ASIANPAINT,3465.66,-17.77,35.0,H-LC,5.73,207548.0,-44220.0,97423.0,0.24,-17.56,46.94,21.13,27.0,-0.45,1.48,11.64,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,51.0,H-LC,3.26,215966.0,5600.0,23173.0,-0.03,2.66,10.73,13.68,37.0,0.24,1.54,19.63,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.01,49.0,H-LC,12.20,282846.0,-63110.0,131354.0,1.19,-18.24,46.44,19.73,1.0,-0.48,2.01,4.18,X40,ATH,IT
40,INFY,2275.00,-18.88,50.0,H-LC,8.02,315346.0,2458.0,169215.0,1.51,0.79,53.66,54.87,3.0,0.01,2.24,7.49,X40,BTT,IT
42,ITC,452.00,-39.61,41.0,H-LC,2.31,195673.0,-4465.0,25359.0,0.09,-2.23,12.96,10.44,4.0,-0.18,1.39,3.21,X40,NTT,FMCG
83,VBL,671.64,-22.30,30.0,H-LC,10.14,279891.0,-35951.0,148622.0,-0.30,-11.38,53.10,35.67,5.0,-0.24,1.99,0.89,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,46.0,H-LC,7.50,249168.0,-12451.0,131660.0,-0.55,-4.76,52.84,45.57,7.0,-0.09,1.77,6.02,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,47.0,L-SC,30.92,80525.0,-33024.0,73076.0,1.35,-29.08,90.75,35.27,268.0,-0.45,0.57,97.09,XR,NTT,HOTELS
49,MASFIN,398.61,-18.70,47.0,H-SC,10.90,92220.0,-5760.0,27362.0,-0.60,-5.88,29.67,22.05,152.0,-0.21,0.66,34.05,XR,ATH,FINANCE
50,NATIONALUM,244.55,-41.18,67.0,H-MC,14.13,117602.0,17098.0,12007.0,2.22,17.01,10.21,28.96,79.0,1.42,0.84,56.76,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,62.0,H-SC,13.48,130194.0,-12285.0,31806.0,-0.41,-8.62,24.43,13.70,163.0,-0.39,0.93,53.66,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,13.58,146268.0,-2952.0,147131.0,-0.35,-1.98,100.59,96.62,208.0,-0.02,1.04,58.29,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,13.58,146268.0,-2952.0,147131.0,-0.35,-1.98,100.59,96.62,208.0,-0.02,1.04,58.29,XY25,NTT,FINANCE
69,SURYODAY,240.00,62.28,62.0,H-SC,7.40,148920.0,-30151.0,95875.0,0.51,-16.84,64.38,36.71,135.0,-0.31,1.06,47.76,XR,NTT,BANKS
26,GREENPANEL,537.00,237.86,62.0,M-SC,5.12,150694.0,-32384.0,111363.0,0.91,-17.69,73.90,43.14,230.0,-0.29,1.07,41.75,XY24,NTT,MISC
32,HINDZINC,730.22,31.71,68.0,M-LC,6.91,214390.0,9314.0,103250.0,0.60,4.54,48.16,54.89,52.0,0.09,1.53,29.73,X5K,ATH,METALS
64,SHALBY,327.00,1230.14,65.0,M-SC,21.52,180804.0,-339.0,45816.0,-0.04,-0.19,25.34,25.10,235.0,-0.01,1.29,44.90,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.06
1,25,43.35
2,50,75.08


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.18
LC    32.17
MC    23.69
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.11
X40      14.61
X40N     11.20
XR       10.98
XY25     10.35
AR        8.13
OX40N     7.68
X200      1.71
MH        1.61
X5K       1.53
SR        1.13
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.06
H-LC    25.64
H-MC    20.44
M-SC    13.70
M-LC     5.51
M-MC     2.95
L-SC     1.42
L-LC     1.02
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.70,-6.57,41.69
IT,12.69,-17.81,81.62
FINANCE,10.60,-13.29,67.99
MISC,6.92,-18.18,83.66
BANKS,6.33,-12.32,71.41
PAINTS,5.54,-18.69,36.38
ELECTRICAL,5.37,-9.24,47.57
INSURANCE,3.82,-3.12,41.67
AUTO,3.49,-17.35,73.45


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3201552.0,22
XR,1368278.0,14
AR,1186950.0,9
X40,788070.0,10
X40N,663228.0,9
OX40N,565828.0,10
XY25,482298.0,7
SR,262055.0,2
X5K,103250.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3709113.0,29
M-SC,1504968.0,17
H-MC,1284114.0,15
H-LC,1190663.0,15
M-LC,376411.0,4
M-MC,368483.0,2
L-SC,260220.0,3
L-MC,59930.0,1
L-LC,41330.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1189591.0      6
M-SC       XY24       843089.0      7
H-SC       AR         814832.0      5
           XR         798552.0      7
H-MC       XY24       569562.0      4
H-LC       X40        501353.0      5
M-MC       XY24       368483.0      2
H-SC       X40N       306730.0      4
           SR         262055.0      2
           OX40N      261056.0      4
H-LC       X40N       220288.0      3
H-MC       X40        213734.0      4
H-LC       AR         203960.0      2
H-MC       XY25       181267.0      2
L-SC       XR         171367.0      2
M-SC       AR         168158.0      2
M-LC       XY24       163754.0      2
M-SC       XY25       147131.0      1
           OX40N      144672.0      4
H-MC       X40N       136210.0      2
M-SC       XR         128935.0      2
H-LC       XY25       112570.0      3
M-LC       XR         109407.0      1
           X5K        103250.0      1
H-MC       XR         100087.0      1
L-SC       OX40N       88853.0      1
H-LC       X200        85419.0      1
H-SC       MH          76297.0      1
M-SC       X40         72983.0      1
H-MC       OX40N       71247.0      1
H-LC       XY24        67073.0      1
L-MC       XR          59930.0      1
L-LC       XY25        41330.0      1
H-MC       MH          12007.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
